In [88]:
%pylab --no-import-all inline
from __future__ import division
import matplotlib.pyplot as plt
import random
import time
import itertools
import urllib
import csv
import math
from LatLon import LatLon


# Points are represented as complex numbers
Point = complex

def X(point): 
    "The x coordinate of a point."
    return point.real

def Y(point): 
    "The y coordinate of a point."
    return point.imag

def distance(A, B): 
    "The distance between two points."
    #print(abs(A - B))
    return abs(A - B)

def Points(n):
    #Defining the set with places, restaurants and hotel.
    
    #TO DO: Method that takes input
    #TO DO: Make difference between node types (hotel, restaurant, places)
    return list([
        ('place',LatLon(48.85837,2.294481000000019)),
        ('place',LatLon(48.864342,2.297820999999999)),
        ('place',LatLon(48.873792,2.295028000000002)),
        ('place',LatLon(48.852968,2.349901999999929)),
        ('place',LatLon(48.860611,2.3376439999999548)),
        #('place',City(48.804865,2.1203550000000178)), #Versailles as outlier
        
        ('restaurant',LatLon(48.860805,2.3005600000000186)),
        ('restaurant',LatLon(48.85955500000001,2.353964000000019)),
        ('restaurant',LatLon(48.867198,289799000000017)),
        
        ('hotel',LatLon(48.88035000000001,2.3173060000000305)),        
        ])
    


def nearest_neighbor(A, points):
    "Find the point in points that is nearest to point A."
    return min(points, key=lambda c: A.distance(c[1]))#distance(c[1], A))
    
    
################################
#Main algorithm to compute the TSP
################################

def nn_tsp_lim(points,days):
    """Start the tour at the first city; at each step extend the tour 
    by moving from the previous city to its nearest neighbor 
    that has not yet been visited."""
    #Number of places to visit per day
    s = math.floor(len(points)/days)
    
    #With this we have at least 2 visits a day
    size = 0
    if s == 1:
        size = s+1
    else:
        size = s
    
    #Has to be the hotel
    start = getHotel(points);
    #We have the same starting point
    tour = [None] * days
    for t in range(0,days):
        tour[t] = [start]
    
    unvisited = list(points)
    unvisited.remove(start) #All unvisited points, places and restaurants together.
    
    prs = placeRestoSplit(unvisited)
    
    places = list(prs[0])
    restos = list(prs[1])
    i = 0
    while unvisited:
        
        if len(tour[i]) <= size:
            
            #it only has the start (hotel) in it for the moment or the previous visited was a restaurant
            if (len(tour[i]) == 1) | (tour[i][-1][0] == 'restaurant'): 
                C = nearest_neighbor(tour[i][-1][1], places)
                places.remove(C)
            
            #Previous places visited was a "place"
            elif tour[i][-1][0] == 'place':
                C = nearest_neighbor(tour[i][-1][1], restos)
                print(restos)
                restos.remove(C)
            
                
            tour[i].append(C)
            
            unvisited.remove(C)
        else:
            i = i+1
            
    return tour

def placeRestoSplit(unvi):
    "split the places and restaurants in the unvisited list"
    
    ans = list(unvi)
    restaurants = []
    for i in range(len(unvi)):
        
        if unvi[i][0] == 'restaurant':
            restaurants.append(unvi[i])
            ans.remove(unvi[i])
            
    return (ans,restaurants)

def getHotel(collection):
    "Search the first elements of tuples to find the hotel"
    #c = set(collection)
    hotelPresent = False
    for hotel in collection:
        
        if hotel[0] == 'hotel':
            return hotel
         
    assert (hotelPresent),"No hotel in the set! Check spelling and if present"


def first(collection):
    "Start iterating over collection, and return the first element."
    return next(iter(collection))

def tour_length(tour):
    "The total of distances between each pair of consecutive cities in the tour."
    return sum(distance(tour[i], tour[i-1]) 
               for i in range(len(tour)))


###################
# Plotting Functions
###################

def plot_mult_tsp(algorithm, points,days):
    "Apply a TSP algorithm to cities, plot the resulting tour, and print information."
    # Find the solution and time how long it takes
    t0 = time.clock()
    tour = algorithm(points,days)
    t1 = time.clock()
    
    plot_tour(tour); 
    plot_var_points(points)
    plt.show()
    
    
def plot_tour(tour): 
    "Plot the cities as circles and the tour as lines between them."
    points = []
    for t in tour:
        points.append(list(t) + [t[0]])
    
    plot_lines(list(points))
    
    
def plot_lines(points):
    "Plot lines to connect a series of points."
    
    colors_ = ['ro-','bo-','go-','mo-','ko-','co-','yo-']
    
    for p,c in zip(points, itertools.cycle(colors_)):
        split_tuples = [i[1] for i in p]
        plt.plot(map(X, split_tuples), map(Y, split_tuples), c)
        
    
    
    plt.axis('scaled'); plt.axis('off')
    

def plot_var_points(points):
    "Plot the different kind of points: places, rest, hotel"
    
    for i in points:
        if i[0] == 'place':
            plt.plot(X(i[1]), Y(i[1]),'ro', markersize=15)
        elif i[0] == 'restaurant':
            plt.plot(X(i[1]), Y(i[1]),'bo', markersize=15)
        else:
            plt.plot(X(i[1]), Y(i[1]),'ro', markersize=15)
            plt.plot(X(i[1]), Y(i[1]),'w+',markeredgewidth=3, markersize=15)
    



Populating the interactive namespace from numpy and matplotlib


In [90]:
plot_mult_tsp(nn_tsp_lim, Points(0),3)

[('restaurant', Latitude 48.860805, Longitude 2.30056), ('restaurant', Latitude 48.859555, Longitude 2.353964), ('restaurant', Latitude 48.867198, Longitude 97.0)]
[('restaurant', Latitude 48.859555, Longitude 2.353964), ('restaurant', Latitude 48.867198, Longitude 97.0)]
[('restaurant', Latitude 48.867198, Longitude 97.0)]


AttributeError: LatLon instance has no attribute 'real'

In [38]:
a = LatLon(48.88035000000001,2.3173060000000305)
b = LatLon(48.804865,2.1203550000000178)
a.distance(b)

16.717055191185803

In [54]:
a = list([
        ('place',LatLon(48.85837,2.294481000000019)),
        ('place',LatLon(48.864342,2.297820999999999)),
        ('place',LatLon(48.873792,2.295028000000002)),
        ('place',LatLon(48.852968,2.349901999999929)),
        ('place',LatLon(48.860611,2.3376439999999548)),
        #('place',City(48.804865,2.1203550000000178)), #Versailles as outlier
        
        ('restaurant',LatLon(48.860805,2.3005600000000186)),
        ('restaurant',LatLon(48.85955500000001,2.353964000000019)),
        ('restaurant',LatLon(48.867198,289799000000017)),
        
        ('hotel',LatLon(48.88035000000001,2.3173060000000305)),        
        ])

hotel = ('hotel',*)
if hotel in a: a.remove(hotel)


SyntaxError: invalid syntax (<ipython-input-54-3e3ef5960a04>, line 16)